# Price Prediction

a project by Felix Marschall, Mika Scheid, Elias Müller and Jan Dorn

## Notebook Structure

1. **Introduction**: Overview of the project and objectives.
2. **Dataset Overview**: Description of the dataset and its key components.
3. **Data Preparation**: 
    - 3.1 Data consolidation
    - 3.2 Data cleaning
    - 3.3 Data transformation
    - 3.4 Data reduction (feature engineering)
4. **Modeling**: Implementation and comparison of different machine learning models.
5. **Hyperparameter Tuning**: Optimization of model parameters.
6. **Evaluation**: Analysis of model performance using various metrics.
7. **Conclusion**: Summary of findings and future work.


## 0. Installing and importing Dependencies

In [ ]:
%pip install pandas
%pip install matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import logging

## 1. Introduction

This project aims to predict Airbnb listing prices using a dataset from Inside Airbnb. The focus is on applying data science techniques to preprocess, analyze, and model the data while exploring additional insights such as time-based patterns and geolocation analysis.

## 2. Dataset Overview

For each location (Amsterdam, Barcelona, Berlin, Budapest, Copenhagen, Istanbul, Lisbon, London, Oslo, Paris, Prague, Rome), various datasets are available from [Inside Airbnb](https://insideairbnb.com/get-the-data/). The most relevant ones are:

- **listings.csv.gz**: Detailed listings data
- **calendar.csv.gz**: Detailed calendar data
- **reviews.csv.gz**: Detailed review data

### **Hypothesis 1**: Reviews are not useful for price prediction

We hypothesize that reviews do not contribute to price prediction, as they are influenced by the price itself. Higher-priced Airbnbs may receive lower ratings if expectations are not met, while cheaper ones may be rated higher due to better perceived value.

Since reviews inherently reflect the price and do not provide independent insights into the accommodation's quality or value, we will exclude them from the initial price prediction model. Instead, we will focus on features like location, property type, size, and amenities, which have a direct impact on the price.

All relevant data for this analysis is available in the **listings.csv.gz** file, which we downloaded for each city from [Inside Airbnb](https://insideairbnb.com/get-the-data/).

### **Hypothesis 2**: Prices remain constant throughout the year

One might assume that Airbnb prices change throughout the year depending on demand. However, this is not the case. 

Upon inspecting the **calendar.csv.gz** file for Amsterdam using SQL (due to the large size of the dataset, SQL is more efficient), we found that prices for every listing remain constant over the entire year.

```sql
SELECT listing_id, COUNT(DISTINCT price) AS num_distinct_prices
FROM calendar
GROUP BY listing_id
ORDER BY num_distinct_prices DESC;
```

SQL Output:

<img src="./images/hypothesis_2.png" alt="SQL Output" width="300"/>

As we can see from the output, no listing in the Amsterdam dataset has more than one price for the same listing throughout the given timeframe. We assume this holds true for every listing in every city, so we do not consider price fluctuations over time in this analysis.


## 3. Data Preparation

Consists of Data Consolidation, Data Cleaning, Data Transformation and Data Reduction




### 3.1 Data Consolidation

The Airbnb listing data for each city is located in `./data/<city_name>/listings.csv.gz`. As a first step, we need to consolidate these datasets into a single dataframe for further analysis.

Checking the columns of each dataset reveals the following:

In [ ]:
data_dir = './data'

for city in os.listdir(data_dir):
    city_path = os.path.join(data_dir, city)
    listings_path = os.path.join(city_path, 'listings.csv.gz')
    if os.path.isdir(city_path) and os.path.exists(listings_path):
        try:
            df = pd.read_csv(listings_path, compression='gzip', nrows=1)
            print(f"{city}: {df.shape[1]} columns")
            # print(f"Columns: {list(df.columns)}\n")
        except Exception as e:
            print(f"Error reading {listings_path}: {e}")

The datasets have different numbers of columns. The datasets with 75 columns are missing the following columns:

- availability_eoy
- number_of_reviews_ly
- estimated_occupancy_l365d
- estimated_revenue_l365d

In the consolidating script, the schema is defined to include all 79 columns. The datasets that are missing these 4 columns will have null values for these fields.

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

data_dir = './data'
all_dfs = []
success = True

# define the desired schema
schema = [
    "id","listing_url","scrape_id","last_scraped","source","name","description","neighborhood_overview","picture_url",
    "host_id","host_url","host_name","host_since","host_location","host_about","host_response_time","host_response_rate",
    "host_acceptance_rate","host_is_superhost","host_thumbnail_url","host_picture_url","host_neighbourhood",
    "host_listings_count","host_total_listings_count","host_verifications","host_has_profile_pic","host_identity_verified",
    "neighbourhood","neighbourhood_cleansed","neighbourhood_group_cleansed","latitude","longitude","property_type",
    "room_type","accommodates","bathrooms","bathrooms_text","bedrooms","beds","amenities","price","minimum_nights",
    "maximum_nights","minimum_minimum_nights","maximum_minimum_nights","minimum_maximum_nights","maximum_maximum_nights",
    "minimum_nights_avg_ntm","maximum_nights_avg_ntm","calendar_updated","has_availability","availability_30",
    "availability_60","availability_90","availability_365","calendar_last_scraped","number_of_reviews",
    "number_of_reviews_ltm","number_of_reviews_l30d","availability_eoy","number_of_reviews_ly",
    "estimated_occupancy_l365d","estimated_revenue_l365d","first_review","last_review","review_scores_rating",
    "review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication",
    "review_scores_location","review_scores_value","license","instant_bookable","calculated_host_listings_count",
    "calculated_host_listings_count_entire_homes","calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms","reviews_per_month"
]

for city in os.listdir(data_dir):
    city_path = os.path.join(data_dir, city)
    listings_path = os.path.join(city_path, 'listings.csv.gz')
    if os.path.isdir(city_path) and os.path.exists(listings_path):
        try:
            df = pd.read_csv(listings_path, compression='gzip', low_memory=False)
            df = df.reindex(columns=schema)
            all_dfs.append(df)
            logging.info(f"{city}: {df.shape[0]} listings loaded from {listings_path}")
        except Exception as e:
            logging.error(f"Failed to load {listings_path}: {e}")
            success = False
    else:
        logging.warning(f"No listings.csv.gz found in {city_path}")

if all_dfs:
    try:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        output_path = os.path.join(data_dir, 'all_listings_consolidated.csv')
        combined_df.to_csv(output_path, index=False, na_rep='null')
        logging.info(f"Successfully combined all listings with defined schema. Final shape: {combined_df.shape}")
        logging.info(f"Combined CSV saved to {output_path}")

        # validation of csv file
        try:
            test_df = pd.read_csv(output_path, low_memory=False)
            logging.info(f"Validation: Successfully read {output_path}. Shape: {test_df.shape}")
        except Exception as e:
            logging.error(f"Validation failed: Could not read {output_path}: {e}")
            success = False

    except Exception as e:
        logging.error(f"Failed to concatenate or save combined DataFrame: {e}")
        success = False
else:
    logging.error("No dataframes to combine. Exiting.")
    success = False

if success:
    logging.info("Data consolidation completed successfully.")
else:
    logging.error("Data consolidation encountered errors.")

In [ ]:
combined_df = pd.read_csv("data/all_listings_consolidated.csv")

In [ ]:
combined_df

In [ ]:
combined_df.info()

### 3.2 Data Cleaning

#### 3.2.1 Handle missing values

Columns with over 40% of missing values can be deleted. Too much missing data to be included in the price prediction model. This is part of 3.4 Data reduction but we already know we don't need these values so let's just do it now.

In [ ]:
# number of missing values per column and percentage
missing_counts = combined_df.isnull().sum()
missing_percent = (missing_counts / len(combined_df)) * 100

# combine both in a dataframe for better overview
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Percent': missing_percent
})

# sort by percentage for better overview in plot
missing_df_sorted = missing_df.sort_values(by='Missing Percent', ascending=True)

# color the bars red if they have more than 40% missing values
bar_colors = ['red' if val > 40 else 'skyblue' for val in missing_df_sorted['Missing Percent']]

# plot
plt.figure(figsize=(12, 10))
plt.barh(missing_df_sorted.index, missing_df_sorted['Missing Percent'], color=bar_colors)
plt.xlabel('Missing Value Percentage')
plt.title('Percentage of Missing Values per Column')
plt.axvline(x=40, color='red', linestyle='--', linewidth=1.5, label='40% threshold')
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# list of columns with more than 40% missing values
columns_to_drop = missing_df[missing_df['Missing Percent'] > 40].index.tolist()

# show
print(f"Columns with more than 40% missing values ({len(columns_to_drop)} total):")
for col in columns_to_drop:
    print(f"- {col}: {missing_df.loc[col, 'Missing Percent']:.2f}%")


Remove these columns:

In [ ]:
combined_df_cleaned = combined_df.drop(columns=columns_to_drop)
combined_df_cleaned.info()

Let's inspect the last remaining columns with missing values, their data type and an example.

In [ ]:
# missing values
missing_counts = combined_df_cleaned.isnull().sum()
missing_percent = (missing_counts / len(combined_df)) * 100

# filter only columns with missing values
missing_columns = missing_counts[missing_counts > 0].index

# list to store the results
rows = []

for col in missing_columns:
    percent_missing = missing_percent[col]
    dtype = combined_df[col].dtype
    example_values = combined_df[col].dropna().unique()
    example = example_values[0] if len(example_values) > 0 else "—"
    
    rows.append({
        "column": col,
        "missing": round(percent_missing, 2),
        "datatype": dtype,
        "example": example
    })

# create dataframe and sort
missing_table = pd.DataFrame(rows)
missing_table = missing_table.sort_values(by="missing", ascending=False)

# show table
print(missing_table.to_string(index=False))


We'll focus on a few key columns for targeted cleaning. For all other columns, missing values will be filled with the label "ML" (Missing Label).

1. Since we're predicting the **price column**, we'll remove any rows where the price is missing (~27%). Although a trained model could estimate these prices, we can't verify their accuracy — and given the amount of available data, these entries don't add much value for now.
2. **beds**: Median
3. **bathrooms**: Median
4. review_scores_value, review_scores_location, review_scores_checkin, review_scores_communication, review_scores_cleanliness, review_scores_accuracy,review_scores_rating: Median or Mean depending on distribution -> check distribution
5. **bedrooms**: Median
6. rest: "ML" or median depending on data type

In [ ]:
# 1. remove rows with missing price
combined_df_cleaned = combined_df_cleaned[combined_df_cleaned['price'].notna()]

# 2. 3. 5.
combined_df_cleaned['beds'] = combined_df_cleaned['beds'].fillna(combined_df_cleaned['beds'].median())
combined_df_cleaned['bathrooms'] = combined_df_cleaned['bathrooms'].fillna(combined_df_cleaned['bathrooms'].median())
combined_df_cleaned['bedrooms'] = combined_df_cleaned['bedrooms'].fillna(combined_df_cleaned['bedrooms'].median())


In [ ]:
# 4. check skewness
print("Skewness review_scores_value:", combined_df['review_scores_value'].skew())
print("Skewness review_scores_location:", combined_df['review_scores_location'].skew())
print("Skewness review_scores_checkin:", combined_df['review_scores_checkin'].skew())
print("Skewness review_scores_communication:", combined_df['review_scores_communication'].skew())
print("Skewness review_scores_cleanliness:", combined_df['review_scores_cleanliness'].skew())
print("Skewness review_scores_accuracy:", combined_df['review_scores_accuracy'].skew())
print("Skewness review_scores_rating:", combined_df['review_scores_rating'].skew())



The skewness of each each column is |col| > 0.5, so we choose the median.

In [ ]:
# 4. fill with median
combined_df_cleaned['review_scores_value'] = combined_df_cleaned['review_scores_value'].fillna(combined_df_cleaned['review_scores_value'].median())
combined_df_cleaned['review_scores_location'] = combined_df_cleaned['review_scores_location'].fillna(combined_df_cleaned['review_scores_location'].median())
combined_df_cleaned['review_scores_checkin'] = combined_df_cleaned['review_scores_checkin'].fillna(combined_df_cleaned['review_scores_checkin'].median())
combined_df_cleaned['review_scores_communication'] = combined_df_cleaned['review_scores_communication'].fillna(combined_df_cleaned['review_scores_communication'].median())
combined_df_cleaned['review_scores_cleanliness'] = combined_df_cleaned['review_scores_cleanliness'].fillna(combined_df_cleaned['review_scores_cleanliness'].median())
combined_df_cleaned['review_scores_accuracy'] = combined_df_cleaned['review_scores_accuracy'].fillna(combined_df_cleaned['review_scores_accuracy'].median())


In [ ]:
# 6. rest: "ML" or median
combined_df_cleaned = combined_df_cleaned.copy()

for col in combined_df_cleaned.columns:
    if pd.api.types.is_numeric_dtype(combined_df_cleaned[col]):
        median_value = combined_df_cleaned[col].median()
        combined_df_cleaned[col] = combined_df_cleaned[col].fillna(median_value)
    else:
        combined_df_cleaned[col] = combined_df_cleaned[col].fillna("ML")

# Prüfung
combined_df_cleaned.info()

#### 3.2.2 Reduce noise

First, identify the outliers. Only applicable on numerical columns. No interpretation of id columns possible.

In [ ]:
# extract numerical columns
numeric_cols = combined_df_cleaned.select_dtypes(include=['number']).columns.tolist()

# prepare number of subplots
num_cols = len(numeric_cols)
cols_per_row = 3
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

# create boxplots
fig, axes = plt.subplots(num_rows, cols_per_row, figsize=(cols_per_row * 5, num_rows * 4))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    axes[i].boxplot(combined_df_cleaned[col].dropna(), vert=False)
    axes[i].set_title(col)
    axes[i].grid(True)

# remove empty axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle("Boxplots der numerischen Spalten (Ausreißererkennung)", fontsize=16, y=1.02)
plt.show()


There are obvious outliers in the columns maximum_nights, minimum_maximum_nights, maximum_maximum_nights and maximum_nights_avg_ntm. We suspect these columns won't be needed in the price prediction models, so we proceed to the data transformation.

### 3.3 Data Transformation

Now we transform the data into a format that is more suitable for modeling. This includes normalizing columns, adjusting data types if needed, discretizing values, and even creating new features.

In [ ]:
combined_df_cleaned.info()

We have identified the need for the following transformations (without normalization):
1. **host_since**, **first_review**, **last_review** needs to be a datetime datatype
2. **host_location** can be divided in city and country
3. **price** needs to be a numerical value -> remove the dollar sign
4. **host_response_rate** and **host_acceptance_rate** should be a numerical value
5. **host_response_time** can be transformed into a numerical representation: within an hour -> 1; within a few hours -> 2; ...

In some of these transformations, we need to set the previously set "ML" values to the median.

In [ ]:
df_transformed = combined_df_cleaned.copy()

# 1. transform date columns to datetime and replace NaT with median
date_cols = ['host_since', 'first_review', 'last_review']
for col in date_cols:
    # convert to datetime
    df_transformed[col] = pd.to_datetime(df_transformed[col], errors='coerce')
    # calculate median
    median_date = df_transformed[col].median()
    # replace NaT with median
    df_transformed[col] = df_transformed[col].fillna(median_date)

# 2. split host_location into host_city and host_country
def split_location(val):
    if isinstance(val, str) and ',' in val:
        parts = val.split(',')
        city = parts[0].strip()
        country = parts[-1].strip()
        return city, country
    else:
        return "ML", "ML"

df_transformed[['host_city', 'host_country']] = df_transformed['host_location'].apply(
    lambda x: pd.Series(split_location(x))
)

# 3. price: remove $-sign and commas, then convert to float
df_transformed['price'] = (
    df_transformed['price']
    .astype(str)
    .str.replace(r"[\$,]", "", regex=True)
    .astype(float)
)

# 4. host_response_rate and host_acceptance_rate: remove % sign, convert to float and replace "ML" with median
for col in ['host_response_rate', 'host_acceptance_rate']:
    # replace % sign and convert to float
    col_series = df_transformed[col].astype(str).str.replace('%', '', regex=True)
    # calculate median
    valid_values = col_series[col_series != 'ML'].astype(float)
    median_val = valid_values.median()
    # replace "ML" with median and convert to float
    df_transformed[col] = col_series.replace('ML', str(median_val)).astype(float)

# 5. host_response_time: transform into numerical representation
df_transformed['host_response_time_num'] = df_transformed['host_response_time'].map({
    'within an hour': 1,
    'within a few hours': 2,
    'within a day': 3,
    'a few days or more': 4,
    'ML': '5'
}).astype(float)

# check result
print("host_since, first_review, last_review: \n", df_transformed[['host_since', 'first_review', 'last_review']].dtypes)
print("host_city, host_country: \n", df_transformed[['host_city', 'host_country']].head())
print("price: \n", df_transformed['price'].head())
print("host_response_rate, host_acceptance_rate: \n", df_transformed[['host_response_rate', 'host_acceptance_rate']].head())
print("host_response_time_num: \n", df_transformed['host_response_time_num'].head())


In [ ]:
df_transformed.info()

### 3.4 Data reduction

Now we want to analyse which features to use for the price prediction model.

In [ ]:
import seaborn as sns

# Correlation analysis for numerical features

# Select only numerical columns and drop non-informative IDs
num_cols = df_transformed.select_dtypes(include=['number']).columns.drop(['id', 'scrape_id', 'host_id'])
corr = df_transformed[num_cols].corr()['price'].sort_values(ascending=False)

print("Top correlated numerical features with price:")
print(corr.drop('price').head(10))
print("\nLeast correlated numerical features with price:")
print(corr.drop('price').tail(10))

# Visualize correlation matrix for top features
top_corr_features = corr.abs().sort_values(ascending=False).head(10).index
plt.figure(figsize=(8, 6))
sns.heatmap(df_transformed[top_corr_features].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of Top Features")
plt.show()

# Check unique values for categorical columns with potential predictive power
cat_cols = [
    'property_type', 'room_type', 'neighbourhood_cleansed', 'host_is_superhost',
    'instant_bookable', 'host_country'
]
for col in cat_cols:
    print(f"\n{col}: {df_transformed[col].nunique()} unique values")
    print(df_transformed[col].value_counts(dropna=False).head())

# Recommendation: 
# - Use highly correlated numerical features (e.g., accommodates, bedrooms, beds, bathrooms, reviews_per_month, etc.)
# - Use categorical features with moderate cardinality (room_type, property_type, neighbourhood_cleansed, host_is_superhost, instant_bookable, host_country)
# - Drop columns with very high cardinality or little predictive value (e.g., name, description, url fields, host_url, picture_url, etc.)